<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/designite/analise_designite_accumulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checa ambiente

In [ ]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
!python3 --version

Python 3.10.12


In [ ]:
!sudo apt install tree > install_tree.log

# Funcões

In [ ]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd accumulo && git rev-list -n 1 $TAG
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [ ]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['ACCUMULO'] = tag + 'accumulo/core/src'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $ACCUMULO -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [ ]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

In [ ]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['2.0.1']
pattern_to_match = r'2.'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['2.0.1']


# Análise Designite

In [ ]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [ ]:
!git clone https://github.com/apache/accumulo.git

Cloning into 'accumulo'...
remote: Enumerating objects: 310333, done.
remote: Counting objects: 100% (3536/3536), done.
remote: Compressing objects: 100% (1343/1343), done.
remote: Total 310333 (delta 1280), reused 3015 (delta 1018), pack-reused 306797
Receiving objects: 100% (310333/310333), 94.55 MiB | 17.44 MiB/s, done.
Resolving deltas: 100% (154602/154602), done.


In [ ]:
!tree accumulo -L 2

accumulo
├── assemble
│   ├── bin
│   ├── conf
│   ├── pom.xml
│   └── src
├── core
│   ├── pom.xml
│   └── src
├── DEPENDENCIES
├── hadoop-mapreduce
│   ├── pom.xml
│   └── src
├── iterator-test-harness
│   ├── pom.xml
│   └── src
├── LICENSE
├── minicluster
│   ├── pom.xml
│   └── src
├── NOTICE
├── pom.xml
├── README.md
├── server
│   ├── base
│   ├── compaction-coordinator
│   ├── compactor
│   ├── gc
│   ├── manager
│   ├── monitor
│   ├── native
│   └── tserver
├── shell
│   ├── pom.xml
│   └── src
├── src
│   └── build
├── start
│   ├── pom.xml
│   └── src
├── test
│   ├── pom.xml
│   └── src
└── TESTING.md

29 directories, 14 files


### Pega a lista de tags

In [ ]:
lista_tags_accumulo = !cd accumulo && git tag
print(f'{len(lista_tags_accumulo)}, {lista_tags_accumulo}')

44, ['1.3.5', '1.3.6', '1.4-development-closed', '1.4.0', '1.4.1', '1.4.2', '1.4.3', '1.4.4', '1.4.5', '1.5.0', '1.5.1', '1.5.2', '1.5.3', '1.5.4', '1.6.0', '1.6.1', '1.6.2', '1.6.3', '1.6.4', '1.7.0', 'rel/1.10.0', 'rel/1.10.1', 'rel/1.10.2', 'rel/1.10.3', 'rel/1.6.5', 'rel/1.6.6', 'rel/1.7.1', 'rel/1.7.2', 'rel/1.7.3', 'rel/1.7.4', 'rel/1.8.0', 'rel/1.8.1', 'rel/1.9.0', 'rel/1.9.1', 'rel/1.9.2', 'rel/1.9.3', 'rel/2.0.0', 'rel/2.0.0-alpha-1', 'rel/2.0.0-alpha-2', 'rel/2.0.1', 'rel/2.1.0', 'rel/2.1.1', 'rel/2.1.2', 'rel/3.0.0']


In [ ]:
pattern_to_extract = r'rel/2'
get_elements_with_pattern(lista_tags_accumulo, pattern_to_extract)

['rel/2.0.0',
 'rel/2.0.0-alpha-1',
 'rel/2.0.0-alpha-2',
 'rel/2.0.1',
 'rel/2.1.0',
 'rel/2.1.1',
 'rel/2.1.2']

In [ ]:
pattern_to_extract = r'rel/3'
get_elements_with_pattern(lista_tags_accumulo, pattern_to_extract)

['rel/3.0.0']

# Análise da tag accumulo_2_0_0

In [ ]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_2_0_0 = get_commit_tag(tag='rel/2.0.0')

['0b34e30c34b7c31ebb84ca6b3b686c1b3e7b19af']


In [ ]:
commit_tag_2_0_0[0]

'0b34e30c34b7c31ebb84ca6b3b686c1b3e7b19af'

In [ ]:
cria_pasta_n_commit(repositorio='accumulo', pasta_n_commit='commit_tag_2_0_0')

[]


[]

In [ ]:
!ls commit_tag_2_0_0

accumulo  DEPENDENCIES		 LICENSE      pom.xml	 shell	test
assemble  hadoop-mapreduce	 minicluster  README.md  src	TESTING.md
core	  iterator-test-harness  NOTICE       server	 start


In [ ]:
checkout_n_commit('commit_tag_2_0_0', commit_tag_2_0_0[0])

Checkout do repositorio no commit 0b34e30c34b7c31ebb84ca6b3b686c1b3e7b19af realizado com sucesso!


In [ ]:
ls -liath commit_tag_2_0_0/accumulo/core/src

total 16K
2890714 drwxr-xr-x 4 root root 4.0K Aug 30 20:23 test/
2889822 drwxr-xr-x 4 root root 4.0K Aug 30 20:23 ./
2889823 drwxr-xr-x 9 root root 4.0K Aug 30 20:23 main/
2889819 drwxr-xr-x 3 root root 4.0K Aug 30 20:23 ../


In [ ]:
!java -jar DesigniteJava.jar -i commit_tag_2_0_0/accumulo/core/src -o commit_tag_2_0_0

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal

In [ ]:
!ls -liat commit_tag_2_0_0/designCodeSmells.csv

2886783 -rw-r--r-- 1 root root 294619 Aug 30 20:41 commit_tag_2_0_0/designCodeSmells.csv


In [ ]:
df_design_smells = pd.read_csv('commit_tag_2_0_0/designCodeSmells.csv')

In [ ]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Unutilized Abstraction
1,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Deficient Encapsulation
2,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Insufficient Modularization
3,core,org.apache.accumulo.core.gc.thrift,Iface,Unutilized Abstraction
4,core,org.apache.accumulo.core.gc.thrift,AsyncIface,Unutilized Abstraction
...,...,...,...,...
3096,core,org.apache.accumulo.core.iterators.system,SourceSwitchingIteratorTest,Unutilized Abstraction
3097,core,org.apache.accumulo.core.iterators.system,TestDataSource,Unutilized Abstraction
3098,core,org.apache.accumulo.core.iterators.system,DeletingIteratorTest,Unutilized Abstraction
3099,core,org.apache.accumulo.core.iterators.system,TimeSettingIteratorTest,Unutilized Abstraction


In [ ]:
df_design_smells['Type Name'].unique()

array(['GCMonitorService', 'Iface', 'AsyncIface', ..., 'TestDataSource',
       'DeletingIteratorTest', 'TimeSettingIteratorTest'], dtype=object)

In [ ]:
df_accumulo_classes_amloc_foc = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/piloto/main/criticos/accumulo/accumulo_arquivos_criticos.csv')
df_accumulo_classes_amloc_foc

,nome,amloc,foc
0,AESCryptoService.java,229,8
1,AbstractInputFormat.java,1906,12
2,AccumuloClient.java,265,8
3,AccumuloConfiguration.java,300,14
4,AccumuloFileOutputFormat.java,466,12
...,...,...,...
183,ZooInfoViewer.java,840,8
184,ZooKeeperInitializer.java,297,11
185,ZooPropStore.java,725,15
186,ZooStore.java,211,18


In [ ]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
lista_classes_criticas = df_accumulo_classes_amloc_foc.nome.to_list()
lista_classes_criticas

['AESCryptoService.java',
 'AbstractInputFormat.java',
 'AccumuloClient.java',
 'AccumuloConfiguration.java',
 'AccumuloFileOutputFormat.java',
 'AccumuloFileOutputFormatIT.java',
 'AccumuloITBase.java',
 'AccumuloInputFormat.java',
 'AccumuloInputFormatIT.java',
 'AccumuloMultiTableInputFormatIT.java',
 'AccumuloOutputFormat.java',
 'AccumuloOutputFormatIT.java',
 'AccumuloRecordReader.java',
 'AccumuloReplicaSystem.java',
 'AccumuloRowInputFormat.java',
 'AccumuloRowInputFormatIT.java',
 'ActiveCompaction.java',
 'Admin.java',
 'AdminUtil.java',
 'Ample.java',
 'AuditMessageIT.java',
 'BatchInputSplit.java',
 'BulkFailureIT.java',
 'BulkImport.java',
 'BulkImporter.java',
 'BulkNewIT.java',
 'ChangeSecret.java',
 'ClassLoaderIT.java',
 'CleanUp.java',
 'CleanUpBulkImport.java',
 'CleanUpIT.java',
 'ClientContext.java',
 'ClientServiceHandler.java',
 'ClusterConfigParser.java',
 'Combiner.java',
 'CompactableImpl.java',
 'CompactableUtils.java',
 'CompactionCoordinator.java',
 'Compac

In [ ]:
lista_classes_criticas_limpas = []
for each in lista_classes_criticas:
  each = each.replace('.java', '')
  lista_classes_criticas_limpas.append(each)

print(lista_classes_criticas_limpas)

['AESCryptoService', 'AbstractInputFormat', 'AccumuloClient', 'AccumuloConfiguration', 'AccumuloFileOutputFormat', 'AccumuloFileOutputFormatIT', 'AccumuloITBase', 'AccumuloInputFormat', 'AccumuloInputFormatIT', 'AccumuloMultiTableInputFormatIT', 'AccumuloOutputFormat', 'AccumuloOutputFormatIT', 'AccumuloRecordReader', 'AccumuloReplicaSystem', 'AccumuloRowInputFormat', 'AccumuloRowInputFormatIT', 'ActiveCompaction', 'Admin', 'AdminUtil', 'Ample', 'AuditMessageIT', 'BatchInputSplit', 'BulkFailureIT', 'BulkImport', 'BulkImporter', 'BulkNewIT', 'ChangeSecret', 'ClassLoaderIT', 'CleanUp', 'CleanUpBulkImport', 'CleanUpIT', 'ClientContext', 'ClientServiceHandler', 'ClusterConfigParser', 'Combiner', 'CompactableImpl', 'CompactableUtils', 'CompactionCoordinator', 'CompactionExecutorsMetrics', 'CompactionIT', 'CompactionManager', 'CompactionService', 'Compactor', 'ConcurrentDeleteTableIT', 'ConditionalWriterIT', 'ConditionalWriterImpl', 'ConfigTransformer', 'ConfigurableCompactionIT', 'CreateTab

In [ ]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas_limpas))

In [ ]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
0,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Unutilized Abstraction
1,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Deficient Encapsulation
2,core,org.apache.accumulo.core.gc.thrift,GCMonitorService,Insufficient Modularization
126,core,org.apache.accumulo.core.clientImpl.thrift,getRootTabletLocation,Unutilized Abstraction
152,core,org.apache.accumulo.core.clientImpl.thrift,getTableConfiguration,Unutilized Abstraction
...,...,...,...,...
3084,core,org.apache.accumulo.core.conf,PropertyTypeTest,Unutilized Abstraction
3085,core,org.apache.accumulo.core.conf,PropertyTypeTest,Insufficient Modularization
3086,core,org.apache.accumulo.core.conf,DeprecatedPropertyUtilTest,Unutilized Abstraction
3087,core,org.apache.accumulo.core.conf,TestPropertyUtil,Broken Hierarchy


In [ ]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
2089,core,org.apache.accumulo.core.clientImpl,ScannerIterator,Cyclic-Dependent Modularization
2092,core,org.apache.accumulo.core.clientImpl,TableOperationsImpl,Cyclic-Dependent Modularization
2100,core,org.apache.accumulo.core.clientImpl,TabletServerBatchReaderIterator,Cyclic-Dependent Modularization
2111,core,org.apache.accumulo.core.clientImpl,ClientContext,Cyclic-Dependent Modularization
2113,core,org.apache.accumulo.core.clientImpl,ClientContext,Hub-like Modularization
2115,core,org.apache.accumulo.core.clientImpl,ScannerImpl,Cyclic-Dependent Modularization
2123,core,org.apache.accumulo.core.clientImpl,ThriftScanner,Cyclic-Dependent Modularization
2131,core,org.apache.accumulo.core.clientImpl,ThriftTransportPool,Cyclic-Dependent Modularization
2142,core,org.apache.accumulo.core.clientImpl,TabletLocator,Cyclic-Dependent Modularization
2146,core,org.apache.accumulo.core.clientImpl,UserCompactionUtils,Cyclic-Dependent Modularization


In [ ]:
df_critical_classes.TypeName.unique()

array(['ScannerIterator', 'TableOperationsImpl',
       'TabletServerBatchReaderIterator', 'ClientContext', 'ScannerImpl',
       'ThriftScanner', 'ThriftTransportPool', 'TabletLocator',
       'UserCompactionUtils', 'Tablet', 'RFile', 'RFileTest', 'TestRFile',
       'TabletsMetadata', 'TabletMetadata', 'Ample', 'FateLogger',
       'AESCryptoService', 'ThriftUtil', 'TabletServerThriftClient',
       'FateThriftClient', 'ManagerThriftClient', 'ThriftClientTypes',
       'Fate', 'ZooCache', 'AccumuloConfiguration'], dtype=object)

In [ ]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('commit_tag_2_0_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('commit_tag_2_0_0/analise_designite.csv', index=False)

In [ ]:
!cd commit_tag_2_0_0 && zip ds_accumulo_2_0_0.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 84%)
  adding: analise_designite.csv (deflated 95%)
  adding: designCodeSmells.csv (deflated 95%)
  adding: implementationCodeSmells.csv (deflated 97%)
  adding: methodMetrics.csv (deflated 94%)
  adding: typeMetrics.csv (deflated 88%)


# Análise da tag v32.1.2


In [ ]:
commit_tag_v32_1_2 = get_commit_tag(tag='v32.1.2')

In [ ]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='guava', pasta_n_commit='guava_32_1_2')

[]

In [ ]:
checkout_n_commit('guava_32_1_2', commit_tag_v32_1_2[0])

Checkout do repositorio no commit db74bd2fdac443223d45e6fc5c66548542be1081 realizado com sucesso!


In [ ]:
!java -jar DesigniteJava.jar -i guava_32_1_2/guava/src/ -o guava_32_1_2

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due 

In [ ]:
df_design_smells_final = pd.read_csv('guava_32_1_2/designCodeSmells.csv')

In [ ]:
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,guava,com.google.thirdparty.publicsuffix,PublicSuffixPatterns,Deficient Encapsulation
1,guava,com.google.common.primitives,UnsignedInteger,Deficient Encapsulation
2,guava,com.google.common.primitives,UnsignedInteger,Insufficient Modularization
3,guava,com.google.common.primitives,UnsignedInts,Insufficient Modularization
4,guava,com.google.common.primitives,SignedBytes,Deficient Encapsulation
...,...,...,...,...
1664,guava,com.google.common.reflect,TypeParameter,Unutilized Abstraction
1665,guava,com.google.common.reflect,TypeParameter,Broken Hierarchy
1666,guava,com.google.common.reflect,TypeToInstanceMap,Unutilized Abstraction
1667,guava,com.google.common.reflect,TypeResolver,Insufficient Modularization


In [ ]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [ ]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [ ]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell
13,guava,com.google.common.primitives,Floats,Cyclic-Dependent Modularization
24,guava,com.google.common.primitives,Doubles,Cyclic-Dependent Modularization
51,guava,com.google.common.primitives,Ints,Cyclic-Dependent Modularization
65,guava,com.google.common.primitives,Longs,Cyclic-Dependent Modularization
72,guava,com.google.common.math,LongMath,Cyclic-Dependent Modularization
...,...,...,...,...
1545,guava,com.google.common.collect,ImmutableList,Cyclic-Dependent Modularization
1619,guava,com.google.common.collect,ImmutableSetMultimap,Cyclic-Dependent Modularization
1627,guava,com.google.common.reflect,ClassPath,Cyclic-Dependent Modularization
1633,guava,com.google.common.reflect,Types,Cyclic-Dependent Modularization


In [ ]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('guava_32_1_2/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('guava_32_1_2/analise_designite.csv', index=False)

In [ ]:
!cd guava_32_1_2 && zip ds_guava_32_1_2.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 88%)
  adding: analise_designite.csv (deflated 92%)
  adding: designCodeSmells.csv (deflated 92%)
  adding: implementationCodeSmells.csv (deflated 94%)
  adding: methodMetrics.csv (deflated 91%)
  adding: typeMetrics.csv (deflated 80%)
